In [ ]:
import sys, os, subprocess, shutil, glob, pathlib

print("Python:", sys.version)
print("Interpreter:", sys.executable)

env_name = os.path.basename(sys.prefix) or "current-env"
print("Detected environment name:", env_name)

req_file = "requirements.txt"
if os.path.exists(req_file):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", req_file])
else:
    subprocess.check_call([sys.executable, "-m", "pip", "install",
                           "numpy==1.23.5", "scipy==1.15.3",
                           "parmap==1.7.0", "pandas==2.3.1",
                           "scikit-learn==1.7.1"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "setuptools>=68", "wheel", "Cython<3.1"])
subprocess.check_call([sys.executable, "-m", "ipykernel", "install",
                       "--user", "--name", env_name, "--display-name", f"Python ({env_name})"])

for p in ["build", "__pycache__"]:
    shutil.rmtree(p, ignore_errors=True)
for f in glob.glob("ns_hmm.*.so") + glob.glob("ns_hmm.*.pyd") + glob.glob("ns_hmm.c"):
    pathlib.Path(f).unlink(missing_ok=True)

for key in ("CFLAGS", "CPPFLAGS", "CXXFLAGS"):
    if key in os.environ:
        os.environ.pop(key)

if os.path.exists("setup.py"):
    print("\n[Build] python setup.py build_ext --inplace ...")
    subprocess.check_call([sys.executable, "setup.py", "build_ext", "--inplace"])
else:
    raise FileNotFoundError("setup.py not found in current directory.")

import importlib
try:
    import ns_hmm
except Exception:
    if "ns_hmm" in sys.modules:
        del sys.modules["ns_hmm"]
    importlib.invalidate_caches()
    import ns_hmm

print("\nSetup complete. ns_hmm loaded:", ns_hmm)
print(f"Jupyter kernel registered as: Python ({env_name})")


In [ ]:
import numpy as np
import scipy as sp
import matplotlib
from matplotlib import pyplot as plt
import math
import seaborn as sns
from matplotlib import gridspec
import multiprocessing
from multiprocessing import get_context
import parmap
import matplotlib.ticker as ticker
from matplotlib.ticker import AutoMinorLocator
from sklearn.mixture import BayesianGaussianMixture as BGMM
import sys, subprocess, shutil, glob, pathlib, os, site, platform, threading
import pickle
print("Python:", sys.version)
print("Interpreter:", sys.executable)
print("Site-packages:", site.getsitepackages())
print("OS:", platform.platform())

In [ ]:
plt.rcParams["font.family"] = "times new roman"
plt.rcParams["mathtext.fontset"] = "cm"
plt.rcParams['figure.dpi'] = 75
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['axes.titlesize'] = 20 

from matplotlib_inline.backend_inline import set_matplotlib_formats

# Nested sampling run

In [ ]:
# user-configurable parameters
user_params = {
    "D_min": 0.001,
    "D_max": 2.0,
    "unit_T": 0.1,
    "unit_L": 0.167,
}
path = 'test_dataset'

ns_hmm.set_params(user_params)
fnames = np.genfromtxt(f'{path}/filenames.txt', dtype=str)
print(ns_hmm.get_params())  # check

In [ ]:
fnum = len(fnames)
findices = np.arange(1, fnum + 1)
num_cores = min(os.cpu_count(), fnum)
indices = np.array_split(findices, num_cores)

print(fnames)
for i in range(fnum):
    globals()[f'trj_{i+1}'] = np.genfromtxt(f'{path}/{fnames[i]}')

def start_logger(ctx):
    mgr = ctx.Manager()
    q = mgr.Queue()
    def _listener():
        for msg in iter(q.get, None):
            print(msg, flush=True)
    t = threading.Thread(target=_listener, daemon=True)
    t.start()
    return q, t, mgr

start_method = "fork" if sys.platform != "win32" else "spawn"
ctx = get_context(start_method)

USE_LOG_QUEUE = True
log_q, t, mgr = (None, None, None)
if USE_LOG_QUEUE:
    log_q, t, mgr = start_logger(ctx)

def _init_worker(params):
    import ns_hmm as _m
    _m.set_params(params)

with ctx.Pool(processes=num_cores, initializer=_init_worker, initargs=(user_params,)) as pool:
    results = parmap.map(
        ns_hmm.mp_nested_sampling,
        indices,
        200, 100000, 0.0001, 0.25, 30, path,
        log_q=log_q,
        pm_pbar=True, pm_pool=pool
    )

if USE_LOG_QUEUE:
    log_q.put(None)
    t.join()


# AIC calculation

In [ ]:
AIC = np.zeros((fnum, 3))
for j in range(fnum):
    temp_AIC = np.inf
    for k in range(3):
        nstates = k + 1
        with open('{}/results/{}totalseries{}state.pkl'.format(path, j+1, nstates), 'rb') as f:
            temp = pickle.load(f)
        temp_MLE = temp[np.argmax(temp[:, nstates**2]), :]
        AIC[j, k] = 2 * (nstates ** 2) - 2 * temp_MLE[nstates ** 2]
        
        if AIC[j, k] < temp_AIC: # model selection
            temp_AIC = AIC[j, k]
            globals()['MLE{}'.format(j+1)] = temp_MLE.copy() # MLE with minimum AIC

with open('{}/results/AIC.pkl'.format(path), 'wb') as f:
    pickle.dump(AIC, f)
        

# Model selection

In [ ]:
model_AIC = np.zeros(fnum, dtype = int)
with open('{}/results/AIC.pkl'.format(path), 'rb') as f:
    IC = pickle.load(f)
model_AIC = np.argmin(IC, axis = 1) + 1
            
model_Bayes = np.zeros(fnum, dtype = int)
for i in range(fnum):
    with open('{}/results/prob{}.pkl'.format(path, i + 1), 'rb') as f:
        prob = pickle.load(f)
    model_Bayes[i] = np.argmax(prob) + 1

In [ ]:
model_AIC

# Parameter estimation (MLE, MAP)

In [ ]:
ns_hmm.NS_Params('uniform', 'AIC', path, fnum)
ns_hmm.NS_Params('uniform', 'Bayesian', path, fnum)

In [ ]:
AIC_MLE_ds = []
AIC_MAP_ds = []
cnt = 0

####--AIC, MLE

for j in range(fnum):
    with open('{}/results/MLE{}_{}.pkl'.format(path, j+1, 'AIC'), 'rb') as f:
        temp = pickle.load(f)
    temp = temp.flatten()
    for k in range(np.sqrt(len(temp)-1).astype(int)):
        AIC_MLE_ds.append(temp[k])
        
####--AIC, MLE

####--AIC, MAP

for j in range(fnum):
    with open('{}/results/MAP{}_{}.pkl'.format(path, j+1, 'AIC'), 'rb') as f:
        temp = pickle.load(f)
    temp = temp.flatten()
    for k in range(np.sqrt(len(temp)-1).astype(int)):
        AIC_MAP_ds.append(temp[k])

####--AIC, MAP

In [ ]:
Bayes_MLE_ds = []
Bayes_MAP_ds = []
cnt = 0

####--Bayes, MLE

for j in range(fnum):
    with open('{}/results/MLE{}_{}.pkl'.format(path, j+1, 'Bayesian'), 'rb') as f:
        temp = pickle.load(f)
    temp = temp.flatten()
    for k in range(np.sqrt(len(temp)-1).astype(int)):
        Bayes_MLE_ds.append(temp[k])
        
####--Bayes, MLE


####--Bayes, MAP

for j in range(fnum):
    with open('{}/results/MAP{}_{}.pkl'.format(path, j+1, 'Bayesian'), 'rb') as f:
        temp = pickle.load(f)
    temp = temp.flatten()
    for k in range(np.sqrt(len(temp)-1).astype(int)):
        Bayes_MAP_ds.append(temp[k])

####--Bayes, MAP

# Parameter clustering

### AIC

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
from sklearn.mixture import BayesianGaussianMixture as BGMM
from scipy.stats import norm
import pickle

def _prep_positive_log10(arr, name="array"):
    arr = np.asarray(arr, dtype=float).ravel()
    arr = arr[np.isfinite(arr) & (arr > 0)]
    if arr.size == 0:
        raise ValueError(f"{name}")
    return np.log10(arr)

def _safe_hist(log10_vals, bins_linspace):
    hist, bins = np.histogram(log10_vals, density=True, bins=bins_linspace)
    # bin center
    binw = bins[1] - bins[0]
    centers = bins[:-1] + binw/2
    return hist, centers, binw

def _boundaries_from_means(mu_log10):
    mu = np.sort(np.asarray(mu_log10).ravel())[::-1]
    mids = (mu[:-1] + mu[1:]) / 2.0
    while len(mids) < 3:
        mids = np.append(mids, mids[-1])
    return mids[:3]

def _bin_index_for_boundary(bound_log10, bin_centers_log10):
    idx = np.searchsorted(bin_centers_log10, bound_log10, side="left")
    return int(np.clip(idx, 0, len(bin_centers_log10)))

# --- plotting params
colors = ['tab:red', 'tab:blue', 'tab:grey', 'tab:grey']

gs = plt.GridSpec(1, 2, wspace=0, width_ratios=[1, 1])
fig = plt.figure(figsize=(2.5, 2.5))
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1], sharey=ax1)

# ===== BGMM, MLE =====
AIC_MLE_ds = np.array(AIC_MLE_ds, dtype=float)
log10_MLE = _prep_positive_log10(AIC_MLE_ds, name="AIC_MLE_ds")

bgmm_mle = BGMM(
    n_components=4,
    max_iter=50,
    n_init=50,
    init_params='kmeans',
    random_state=1,
    weight_concentration_prior=20,
    mean_precision_prior=0.5,
    covariance_type='spherical',
    covariance_prior=0.3,
    degrees_of_freedom_prior=5
).fit(log10_MLE[:, None])

labels_mle = bgmm_mle.predict(log10_MLE[:, None])
mu_mle = bgmm_mle.means_.ravel()
var_mle = bgmm_mle.covariances_.ravel()
w_mle = bgmm_mle.weights_.ravel()

for i in range(4):
    count_i = np.count_nonzero(labels_mle == i)
    print(count_i)
    print('weight:', w_mle[i])
    print('mu:', 10**mu_mle[i])
    print('var:', var_mle[i])

bins_lin = np.linspace(-3, 0.5, 35)
hist_mle, centers_mle, binw_mle = _safe_hist(log10_MLE, bins_lin)

bounds_mle_log10 = _boundaries_from_means(mu_mle)
index1 = _bin_index_for_boundary(bounds_mle_log10[0], centers_mle)
index2 = _bin_index_for_boundary(bounds_mle_log10[1], centers_mle)
index3 = _bin_index_for_boundary(bounds_mle_log10[2], centers_mle)

yvals_mle = 10**centers_mle
hscale_mle = binw_mle * 2.3 * yvals_mle

ax1.barh(yvals_mle[index1:], hist_mle[index1:], height=hscale_mle[index1:], color=colors[0], alpha=0.4)
ax1.barh(yvals_mle[index2:index1], hist_mle[index2:index1], height=hscale_mle[index2:index1], color=colors[1], alpha=0.4)
ax1.barh(yvals_mle[index3:index2], hist_mle[index3:index2], height=hscale_mle[index3:index2], color=colors[2], alpha=0.4)
ax1.barh(yvals_mle[:index3], hist_mle[:index3], height=hscale_mle[:index3], color=colors[3], alpha=0.4)

ax1.invert_xaxis()
ax1.set_yscale('log')
ax1.set_ylim(0.0005, 5)
ax1.set_title('MLE')

xspace2 = np.linspace(1, 0, 5)
for i in range(3):
    ax1.plot(xspace2, np.full_like(xspace2, 10**np.sort(mu_mle)[::-1][i]), '--', linewidth=2, color=colors[i])

# ===== BGMM, MAP =====
AIC_MAP_ds = np.array(AIC_MAP_ds, dtype=float)
log10_MAP = _prep_positive_log10(AIC_MAP_ds, name="AIC_MAP_ds")

bgmm_map = BGMM(
    n_components=4,
    max_iter=50,
    n_init=50,
    init_params='kmeans',
    random_state=1,
    weight_concentration_prior=20,
    mean_precision_prior=0.5,
    covariance_type='spherical',
    covariance_prior=0.3,
    degrees_of_freedom_prior=5
).fit(log10_MAP[:, None])

labels_map = bgmm_map.predict(log10_MAP[:, None])
mu_map = bgmm_map.means_.ravel()
var_map = bgmm_map.covariances_.ravel()
w_map = bgmm_map.weights_.ravel()

for i in range(4):
    count_i = np.count_nonzero(labels_map == i)
    print(count_i)
    print('weight:', w_map[i])
    print('mu:', 10**mu_map[i])
    print('var:', var_map[i])

hist_map, centers_map, binw_map = _safe_hist(log10_MAP, bins_lin)
bounds_map_log10 = _boundaries_from_means(mu_map)
index1_map = _bin_index_for_boundary(bounds_map_log10[0], centers_map)
index2_map = _bin_index_for_boundary(bounds_map_log10[1], centers_map)
index3_map = _bin_index_for_boundary(bounds_map_log10[2], centers_map)

yvals_map = 10**centers_map
hscale_map = binw_map * 2.3 * yvals_map

ax2.barh(yvals_map[index1_map:], hist_map[index1_map:], height=hscale_map[index1_map:], color=colors[0], alpha=0.4)
ax2.barh(yvals_map[index2_map:index1_map], hist_map[index2_map:index1_map], height=hscale_map[index2_map:index1_map], color=colors[1], alpha=0.4)
ax2.barh(yvals_map[index3_map:index2_map], hist_map[index3_map:index2_map], height=hscale_map[index3_map:index2_map], color=colors[2], alpha=0.4)

ax2.set_yscale('log')
ax2.yaxis.tick_right()
ax2.set_title('MAP')
ax2.set_xlabel('PDF')
ax2.xaxis.set_label_coords(0, -0.15)
ax1.yaxis.set_label_coords(-0.4, 0.5)
ax1.set_ylabel(r'$D~(\mu \mathrm{m^2 / s})$')

for i in range(3):
    ax2.plot(xspace2, np.full_like(xspace2, 10**np.sort(mu_map)[::-1][i]), '--', linewidth=2, color=colors[i])

max_pdf_val_mle = max(
    hist_mle[index1:].max(initial=0),
    hist_mle[index2:index1].max(initial=0),
    hist_mle[index3:index2].max(initial=0),
    hist_mle[:index3].max(initial=0)
)
ax1.set_xlim(max_pdf_val_mle * 1.1, 0)

max_pdf_val_map = max(
    hist_map[index1_map:].max(initial=0),
    hist_map[index2_map:index1_map].max(initial=0),
    hist_map[index3_map:index2_map].max(initial=0)
)
ax2.set_xlim(0, max_pdf_val_map * 1.1)

plt.show()

P_avg = np.zeros((2, 2))
cnt = 0

for j in range(int(fnum)):
    pkl_path = f'{path}/results/MAP{j+1}_AIC.pkl'
    try:
        with open(pkl_path, 'rb') as f:
            temp = pickle.load(f)
        win = int(np.sqrt(max(len(temp) - 1, 1)))
        tempD, tempP = ns_hmm.sample_sorting(temp, win)
        # P_avg += tempP; cnt += 1
    except FileNotFoundError:
        print(f"path not found: {pkl_path}")
    except Exception as e:
        print(f"exception ({pkl_path}): {e}")

# if cnt > 0:
#     P_avg /= cnt

# ===== ticks & locators
for ax in (ax1, ax2):
    ax.tick_params(which='major', labelsize=15, direction='in', length=7, bottom=True)
    ax.tick_params(which='minor', direction='in', length=5)
    ax.set_yticks([0.001, 0.01, 0.1, 1])
    ax.yaxis.set_minor_locator(ticker.LogLocator(base=10.0,
        subs=(0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9), numticks=10))

# plt.savefig('AIC_clustering.pdf', format='pdf', dpi=1200)


### Bayesian

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
from sklearn.mixture import BayesianGaussianMixture as BGMM
import pickle

# ===== helpers =====
def _prep_log10_pos(arr, name):
    arr = np.asarray(arr, float).ravel()
    arr = arr[np.isfinite(arr) & (arr > 0)]
    if arr.size == 0:
        raise ValueError(f"{name}")
    return np.log10(arr)

def _safe_hist(log10_vals, bins_lin):
    hist, bins = np.histogram(log10_vals, density=True, bins=bins_lin)
    w = bins[1] - bins[0]
    centers = bins[:-1] + w/2
    return hist, centers, w

def _bounds_from_means_linear(mu_log10, k=3):
    mu = np.sort(np.asarray(mu_log10).ravel())[::-1]
    mids = (mu[:-1] + mu[1:]) / 2.0
    if mids.size == 0:
        return np.array([np.nan, np.nan, np.nan])
    while mids.size < k:
        mids = np.append(mids, mids[-1])
    return (10**mids[:k]).astype(float)

def _seg_max(hist, i0=None, i1=None):
    h = hist if (i0 is None and i1 is None) else hist[i0:i1]
    return 0.0 if h.size == 0 else float(np.max(h))

# ===== figure =====
gs = plt.GridSpec(1, 2, wspace=0, width_ratios=[1, 1])
fig = plt.figure(figsize=(2.5, 2.5))
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1], sharey=ax1)

# ===== MLE =====
log10_mle = _prep_log10_pos(Bayes_MLE_ds, "Bayes_MLE_ds")

bgmm_mle = BGMM(
    n_components=4, max_iter=50, n_init=20, init_params='kmeans',
    random_state=1, weight_concentration_prior=20, mean_precision_prior=0.5,
    covariance_type='spherical', covariance_prior=0.3, degrees_of_freedom_prior=5
).fit(log10_mle[:, None])

labels_mle = bgmm_mle.predict(log10_mle[:, None])
mu_mle = bgmm_mle.means_.ravel()
var_mle = bgmm_mle.covariances_.ravel()
w_mle = bgmm_mle.weights_.ravel()

for i in range(4):
    print(np.count_nonzero(labels_mle == i))
    print('weight:', w_mle[i])
    print('mu:', 10**mu_mle[i])
    print('var:', var_mle[i])

bins_lin = np.linspace(-3, 0.5, 35)
hist_mle, centers_mle, bw_mle = _safe_hist(log10_mle, bins_lin)
yvals_mle = 10**centers_mle
hscale_mle = bw_mle * 2.3 * yvals_mle

bounds_mle = _bounds_from_means_linear(mu_mle, k=3)
bound1, bound2, bound3 = bounds_mle
globals()['bound1'] = bound1
globals()['bound2'] = bound2
globals()['bayesMLEbound1'] = bound1
globals()['bayesMLEbound2'] = bound2
globals()['bayesMLEbound3'] = bound3

index1 = int(np.searchsorted(yvals_mle, bound1, side='left'))
index2 = int(np.searchsorted(yvals_mle, bound2, side='left'))
index3 = int(np.searchsorted(yvals_mle, bound3, side='left'))

# barh
ax1.barh(yvals_mle[index1:], hist_mle[index1:], height=hscale_mle[index1:], color=colors[0], alpha=0.4)
ax1.barh(yvals_mle[index2:index1], hist_mle[index2:index1], height=hscale_mle[index2:index1], color=colors[1], alpha=0.4)
ax1.barh(yvals_mle[index3:index2], hist_mle[index3:index2], height=hscale_mle[index3:index2], color=colors[2], alpha=0.4)
# ax1.barh(yvals_mle[:index3], hist_mle[:index3], height=hscale_mle[:index3], color=colors[3], alpha=0.4)

# 축/스타일
ax1.invert_xaxis()
ax1.set_yscale('log')
ax1.set_ylim(0.0005, 5)
ax1.set_title('MLE')
ax1.tick_params(which='both', direction='in')

xspace2 = np.linspace(1, 0, 5)
for i in range(3):
    ax1.plot(xspace2, np.full_like(xspace2, 10**np.sort(mu_mle)[::-1][i]), '--', linewidth=2, color=colors[i])

max_pdf_val_mle = max(
    _seg_max(hist_mle, index1, None),
    _seg_max(hist_mle, index2, index1),
    _seg_max(hist_mle, index3, index2),
    _seg_max(hist_mle, None, index3),
)
ax1.set_xlim(max_pdf_val_mle * 1.1, 0)

# ===== MAP =====
log10_map = _prep_log10_pos(Bayes_MAP_ds, "Bayes_MAP_ds")

bgmm_map = BGMM(
    n_components=4, max_iter=50, n_init=20, init_params='kmeans',
    random_state=1, weight_concentration_prior=20, mean_precision_prior=0.5,
    covariance_type='spherical', covariance_prior=0.3, degrees_of_freedom_prior=5
).fit(log10_map[:, None])

labels_map = bgmm_map.predict(log10_map[:, None])
mu_map = bgmm_map.means_.ravel()
var_map = bgmm_map.covariances_.ravel()
w_map = bgmm_map.weights_.ravel()

for i in range(4):
    print(np.count_nonzero(labels_map == i))
    print('weight:', w_map[i])
    print('mu:', 10**mu_map[i])
    print('var:', var_map[i])

hist_map, centers_map, bw_map = _safe_hist(log10_map, bins_lin)
yvals_map = 10**centers_map
hscale_map = bw_map * 2.3 * yvals_map

bounds_map = _bounds_from_means_linear(mu_map, k=3)
bayesMAPbound1, bayesMAPbound2, bayesMAPbound3 = bounds_map
globals()['bayesMAPbound1'] = bayesMAPbound1
globals()['bayesMAPbound2'] = bayesMAPbound2
globals()['bayesMAPbound3'] = bayesMAPbound3

index1_map = int(np.searchsorted(yvals_map, bayesMAPbound1, side='left'))
index2_map = int(np.searchsorted(yvals_map, bayesMAPbound2, side='left'))
index3_map = int(np.searchsorted(yvals_map, bayesMAPbound3, side='left'))

ax2.barh(yvals_map[index1_map:], hist_map[index1_map:], height=hscale_map[index1_map:], color=colors[0], alpha=0.4)
ax2.barh(yvals_map[index2_map:index1_map], hist_map[index2_map:index1_map], height=hscale_map[index2_map:index1_map], color=colors[1], alpha=0.4)
ax2.barh(yvals_map[index3_map:index2_map], hist_map[index3_map:index2_map], height=hscale_map[index3_map:index2_map], color=colors[2], alpha=0.4)

ax2.set_yscale('log')
ax2.yaxis.tick_right()
ax2.set_title('MAP')
ax2.set_xlabel('PDF')
ax2.xaxis.set_label_coords(0, -0.15)
ax2.tick_params(which='both', direction='in')

for i in range(3):
    ax2.plot(xspace2, np.full_like(xspace2, 10**np.sort(mu_map)[::-1][i]), '--', linewidth=2, color=colors[i])

max_pdf_val_mle = max(
    hist_mle[index1:].max(initial=0),
    hist_mle[index2:index1].max(initial=0),
    hist_mle[index3:index2].max(initial=0),
    hist_mle[:index3].max(initial=0)
)
ax1.set_xlim(max_pdf_val_mle * 1.1, 0)

max_pdf_val_map = max(
    hist_map[index1_map:].max(initial=0),
    hist_map[index2_map:index1_map].max(initial=0),
    hist_map[index3_map:index2_map].max(initial=0)
)
ax2.set_xlim(0, max_pdf_val_map * 1.1)

plt.show()

# ===== average transition matrix =====
P_avg = np.zeros((2, 2))
cnt = 0
for j in range(int(fnum)):
    pkl_path = f'{path}/results/MAP{j+1}_AIC.pkl'
    try:
        with open(pkl_path, 'rb') as f:
            temp = pickle.load(f)
        win = int(np.sqrt(max(len(temp)-1, 1)))
        tempD, tempP = ns_hmm.sample_sorting(temp, win)
        if len(tempD) == 2:
            if (globals()['bound1'] <= tempD[1]) and (globals()['bound2'] <= tempD[0] < globals()['bound1']):
                P_avg += tempP
                cnt += 1
    except FileNotFoundError:
        print(f"path not found: {pkl_path}")
    except Exception as e:
        print(f"exception ({pkl_path}): {e}")

if cnt > 0:
    P_avg /= cnt
# print(P_avg)
